# Chapter 9: DL for IoT Healthcare 

# Use Case 2:  IoT for Acne detection and Care

# Model: MobileNet V1 (CNN)

# Step 1: Data Collection (downloading)

We do not have any dataset for this use case. We will scrap skin disease images from http://www.dermnet.com/

# Image Scraper 

In [1]:
# Imports the modules
import requests
from bs4 import BeautifulSoup
import os
import shutil

# Root URL 

root_URL = 'http://www.dermnet.com/'

# Skin diseases types 
type_name = ['Acne-and-Rosacea-Photos', 'Actinic-Keratosis-Basal-Cell-Carcinoma-and-other-Malignant-Lesions', 
             'Atopic-Dermatitis-Photos', 'Bullous-Disease-Photos', 'Cellulitis-Impetigo-and-other-Bacterial-Infections', 
             'Eczema-Photos', 'Exanthems-and-Drug-Eruptions', 'Hair-Loss-Photos-Alopecia-and-other-Hair-Diseases',
             'Herpes-HPV-and-other-STDs-Photos', 'Light-Diseases-and-Disorders-of-Pigmentation',
             'Lupus-and-other-Connective-Tissue-diseases', 'Melanoma-Skin-Cancer-Nevi-and-Moles', 'Nail-Fungus-and-other-Nail-Disease', 
             'Poison-Ivy-Photos-and-other-Contact-Dermatitis', 'Psoriasis-pictures-Lichen-Planus-and-related-diseases', 
             'Scabies-Lyme-Disease-and-other-Infestations-and-Bites', 'Seborrheic-Keratoses-and-other-Benign-Tumors', 
             'Systemic-Disease', 'Tinea-Ringworm-Candidiasis-and-other-Fungal-Infections', 
             'Urticaria-Hives', 'Vascular-Tumors', 'Vasculitis-Photos', 'Warts-Molluscum-and-other-Viral-Infections']


type_LinksA = []
type_PagesA = []
type_SubLinksA = []

def find_max(link_):
    r_ = requests.get(link_)
    html_page1_1 = r_.text
    soup_page1_1 = BeautifulSoup(html_page1_1, "html5lib")

    navigationL = soup_page1_1.find_all("div", attrs={"class": "pagination"})
    max_ = 1
    last_ =1

    if not navigationL:
        pass
    else:
        for navi_ in navigationL: 
            for i in navi_.children:
                try:
                    i_ = i.contents[0]
                except:
                    pass
                else:
                    if i_ == 'Next':
                        max_ = int(last_)
                        break
                    else: 
                        last_ = i_
    return int(max_)

# Function to convert images to downloadble link for the images

def images2links(imagesL):
    thumbRLinks = []
    
    for url_ in imagesL:
        
        soup_page1_1_ = BeautifulSoup(requests.get(url_).text, "html5lib")

        for link in soup_page1_1_.find_all("img"):
            link_ = link.get("src")
            if 'Thumb' in link_:
                thumbRLinks.append(link_.replace('Thumb', ''))
    return list(set(thumbRLinks))

def download(url, image_path):
    try:
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(image_path, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
    except Exception as e:
        print (e)
        print ("Failed to save " + image_path)
        print (url + "\n")

    else:
        print ("Successfully saved " + image_path)


# Call the above functions and download the images

In [ ]:
# Main function 
def main():
    for sub_ in type_name:
        SubLinks = []        
        r_ = requests.get(root_URL + '/images/' + sub_)
        html_page1 = r_.text
        soup_page1 = BeautifulSoup(html_page1, "html5lib")

        for link in soup_page1.find_all('a'):
            id_ = link.get('href')
            if '/images/' in id_:
                
                SubLinks.append(root_URL + id_)
        type_LinksA.append(SubLinks)

    attr = '/photos/'

    for type_ in type_LinksA:
        maxL_ = []
        thumbRLinks = []
        
        for link_ in type_:
            imagesL = []
            max_ = find_max(link_)
            maxL_.append(max_)
            print (link_)
            for i_ in range(max_):
                imagesL.append(link_ + attr + str(i_+1))
            
            print ("Pages: ", len(imagesL))
            realLinks = images2links(imagesL)
            print ("Links: ", len(realLinks))
            thumbRLinks.append(realLinks)

        type_PagesA.append(maxL_)
        type_SubLinksA.append(thumbRLinks)

    dir_root_stored_images = './DermnetDataset/'

    if os.path.exists(dir_root_stored_images):
        pass    
    else:
        os.mkdir(dir_root_stored_images) 
    for category in type_name:
        iloc = type_name.index(category)
        dir_disease = dir_root_stored_images + category + '/'
        
        if os.path.exists(dir_disease):
            pass    
        else:
            os.mkdir(dir_disease) 
            
        print ("Disease: ", dir_disease)
        for sub_ in type_LinksA[iloc]:
            iloc_sub = type_LinksA[iloc].index(sub_)
           
            dir_sub_disease = dir_disease + sub_.split('/')[-1]
            
            
            if os.path.exists(dir_sub_disease):
                pass    
            else:
                os.mkdir(dir_sub_disease)
            
            for l_ in type_SubLinksA[iloc][iloc_sub]:
                img_path = dir_sub_disease + '/' + l_.split('/')[-1]
                img_url = l_
                download(img_url, img_path)

if __name__ == "__main__":
    main()

# Step 2: Pre-processing

# Step 3: Model Training and Validation

In [ ]:
Download the mobilenet_1.0_224 model from https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet_v1.
md, and save it into a dirctory that will be your model directory in the following code (we used Mobilenet)

In [ ]:


import os
import argparse
import tensorflow as tf
from IPython.display import Image

import retrain  # Available in the usecase 2 folder

In [34]:
# Have a look at the default values
# For the meaning of these values look at retrain_CNN.py:

FLAGS = argparse.Namespace()
FLAGS.image_dir = ""
FLAGS.output_graph = '.\\retrained_graph.pb'
FLAGS.output_labels = '.\\output_labels.txt'
FLAGS.summaries_dir = '.\\summaries'
FLAGS.how_many_training_steps = 4000
FLAGS.learning_rate = 0.01
FLAGS.testing_percentage = 10
FLAGS.validation_percentage = 10
FLAGS.eval_step_interval = 10
FLAGS.train_batch_size = 100
FLAGS.test_batch_size = -1
FLAGS.validation_batch_size = 100
FLAGS.intermediate_store_frequency =10
FLAGS.print_misclassified_test_images = False
FLAGS.model_dir = "."
FLAGS.bottleneck_dir = "bottlenecks"
FLAGS.final_tensor_name = "final_result"
FLAGS.flip_left_right = False
FLAGS.random_crop = 0
FLAGS.random_scale = 0
FLAGS.random_brightness = 0
FLAGS.intermediate_output_graphs_dir ="."
FLAGS.architecture = "mobilenet_1.0_224"

In [36]:
# change default: 
FLAGS.how_many_training_steps = 500
FLAGS.model_dir = "Mobilenet"
# FLAGS.summaries_dir = "C:\\to\\temp" 
FLAGS.output_graph = "retrained_graph_v2.pb"  
FLAGS.output_labels = "retrained_labels.txt"
FLAGS.image_dir = "dataset-Acne-reduced"


In [37]:
retrain.FLAGS = FLAGS
tf.app.run(main=retrain.main)

INFO:tensorflow:Looking for images in 'Hidradenitis-Suppurativa'
INFO:tensorflow:Looking for images in 'Rosacea'
INFO:tensorflow:Looking for images in 'Perioral-Dermatitis'
INFO:tensorflow:Looking for images in 'Acne-Cystic'
INFO:tensorflow:100 bottleneck files created.
INFO:tensorflow:200 bottleneck files created.
INFO:tensorflow:300 bottleneck files created.
INFO:tensorflow:400 bottleneck files created.
INFO:tensorflow:500 bottleneck files created.
INFO:tensorflow:2019-05-17 18:28:05.404075: Step 0: Train accuracy = 62.0%
INFO:tensorflow:2019-05-17 18:28:05.405044: Step 0: Cross entropy = 1.240135
INFO:tensorflow:2019-05-17 18:28:06.605404: Step 0: Validation accuracy = 52.0% (N=100)
INFO:tensorflow:2019-05-17 18:28:07.176032: Step 10: Train accuracy = 64.0%
INFO:tensorflow:2019-05-17 18:28:07.177150: Step 10: Cross entropy = 3.137362
INFO:tensorflow:2019-05-17 18:28:07.248158: Step 10: Validation accuracy = 56.0% (N=100)
INFO:tensorflow:Save intermediate result to : .intermediate_10

INFO:tensorflow:Converted 2 variables to const ops.
INFO:tensorflow:2019-05-17 18:28:30.334216: Step 200: Train accuracy = 98.0%
INFO:tensorflow:2019-05-17 18:28:30.335376: Step 200: Cross entropy = 0.067607
INFO:tensorflow:2019-05-17 18:28:30.545842: Step 200: Validation accuracy = 89.0% (N=100)
INFO:tensorflow:Save intermediate result to : .intermediate_200.pb
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
INFO:tensorflow:2019-05-17 18:28:31.738629: Step 210: Train accuracy = 100.0%
INFO:tensorflow:2019-05-17 18:28:31.740039: Step 210: Cross entropy = 0.026206
INFO:tensorflow:2019-05-17 18:28:31.804317: Step 210: Validation accuracy = 90.0% (N=100)
INFO:tensorflow:Save intermediate result to : .intermediate_210.pb
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
INFO:tensorflow:2019-05-17 18:28:32.763361: Step 220: Train accuracy = 99.0%
INFO:tensorflow:2019-05-17 18:28:32.764587: Step 220: Cross entropy = 

INFO:tensorflow:2019-05-17 18:28:54.148880: Step 400: Validation accuracy = 89.0% (N=100)
INFO:tensorflow:Save intermediate result to : .intermediate_400.pb
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
INFO:tensorflow:2019-05-17 18:28:55.046226: Step 410: Train accuracy = 100.0%
INFO:tensorflow:2019-05-17 18:28:55.047365: Step 410: Cross entropy = 0.017684
INFO:tensorflow:2019-05-17 18:28:55.102854: Step 410: Validation accuracy = 95.0% (N=100)
INFO:tensorflow:Save intermediate result to : .intermediate_410.pb
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
INFO:tensorflow:2019-05-17 18:28:55.992305: Step 420: Train accuracy = 100.0%
INFO:tensorflow:2019-05-17 18:28:55.993422: Step 420: Cross entropy = 0.017807
INFO:tensorflow:2019-05-17 18:28:56.051786: Step 420: Validation accuracy = 94.0% (N=100)
INFO:tensorflow:Save intermediate result to : .intermediate_420.pb
INFO:tensorflow:Froze 2 variables.
INFO:t

SystemExit: 

# Step 4: Test model on new picture individual Acne Images in Raspberry Pi or Other Devices.

In [53]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np
import tensorflow as tf
# Function to load a pretrained model (transfer learning)

def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

# Function for image to tensor conversion

def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
  input_name = "file_reader"
  #output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
  else:
    image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0)
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result

# Function for loading label files

def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label


In [54]:
# Provide input to be predicted with model file and label or class file 
file_name = "test_images/Acne_test1.jpg"
model_file = "retrained_graph_v2.pb"
label_file = "retrained_labels.txt"

# Fixed input for Mobilenet V1
input_height = 224
input_width = 224
input_mean = 0
input_std = 255
input_layer = "input"
output_layer = "final_result"

# Load the model
graph = load_graph(model_file)
t = read_tensor_from_image_file(
      file_name,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)

input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })

results = np.squeeze(results)

top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)
for i in top_k:
    print("Pontenially", labels[i], "with confidence level",results[i])

Pontenially acne cystic with confidence 0.9969146
Pontenially hidradenitis suppurativa with confidence 0.0019573767
Pontenially rosacea with confidence 0.0011249987
Pontenially perioral dermatitis with confidence 2.9915207e-06
